### Authentication by encrypted negative password

## Authentication

In [9]:
import os
import binascii
import json
import base64
import hashlib
from Crypto import Random
from Crypto.Cipher import AES
import random

##### converting the given password into bytes

In [10]:
password = "pavaniMandadi8"
byte_pass = bytes(password, "utf-8")
byte_pass

b'pavaniMandadi8'

##### Generating the Hex digit and binary digit of the given byte string

In [11]:
hash_256 = hashlib.sha256()
hash_256.update(byte_pass)
hex_digit = hash_256.hexdigest()
binary_digit = "{0:08b}".format(int(hex_digit, 16))
print("Hex Digit of given password :",hex_digit)
print("Binary Digit of given password :",binary_digit)

Hex Digit of given password : c290f15db1f4c02480186dc96d2a35622e04e116520f3502023a8affa9738d6d
Binary Digit of given password : 1100001010010000111100010101110110110001111101001100000000100100100000000001100001101101110010010110110100101010001101010110001000101110000001001110000100010110010100100000111100110101000000100000001000111010100010101111111110101001011100111000110101101101


##### FInding the Nth permutation of the binary digit - N is the length of the given password string

In [12]:
def next_permutation(L): 
    n = len(L) 
    i = n - 2
    while i >= 0 and L[i] >= L[i + 1]: 
        i -= 1
    if i == -1: 
        return False
    j = i + 1
    while j < n and L[j] > L[i]: 
        j += 1
    j -= 1
    L[i], L[j] = L[j], L[i] 
    left = i + 1
    right = n - 1
    while left < right: 
        L[left], L[right] = L[right], L[left] 
        left += 1
        right -= 1
    return True

def nPermute(string, n): 
    string = list(string) 
    new_string = []
    j = 2
    # we reach nth position 
    while next_permutation(string): 
        new_string = string 
  
        # check for nth iteration 
        if j == n: 
            break
        j += 1
  
    # print string after nth iteration 
    return ''.join(new_string)

n_permutated_str = nPermute(binary_digit, len(password)) 

print("The % Permutation of binary digit is : %s", len(password), n_permutated_str)

The % Permutation of binary digit is : %s 14 1100001010010000111100010101110110110001111101001100000000100100100000000001100001101101110010010110110100101010001101010110001000101110000001001110000100010110010100100000111100110101000000100000001000111010100010101111111110101001011100111000110110100111


##### Generating negative database with sequence of symbols

In [42]:
symbols = {"00": "0", "01": "1", "11": "*", "10": "*"}
sequence_of_symbols = "".join([symbols[n_permutated_str[each: each + 2]] for each in range(0, len(n_permutated_str)) if each < len(n_permutated_str)-1])
sequence_of_symbols

'**0001*1*01*0001****001*1*1***1**1**001*****1*01**00000001*01*01*0000000001**0001**1**1***01*01*1**1**1*01*1*1*001**1*1*1**001*001*1***000001*01***0001*001*1**01*1*01*00001****01**1*1*000001*0000001*001***1*1*001*1*1*********1*1*01*1***01***001**1**1*01**'

##### Encrypting Negative database using AES

In [17]:
BLOCK_SIZE = 16
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)


def encrypt(raw, password):
    private_key = hashlib.sha256(password.encode("utf-8")).digest()
    raw = pad(raw)
    iv = Random.new().read(AES.block_size)
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return base64.b64encode(iv + cipher.encrypt(raw))

encoded_string = encrypt(sequence_of_symbols, password)

print("AES Encrypted String :", encoded_string)

AES Encrypted String : b'WNvbSzwBQWtfEyJYwrkOiUM8YTJLBGI0VbdhwYpCjzVyEVCmOMRDPmbK1XcvRgI3iKc69JCBVeCNf96Kv5ilaZoA9JE6JT6yYiHykxU0ajayD18DxAMdZ2RW6T7pDO7vGZqFiLcydRtb2TgoNXK3xRKkZjxpRKFRfsB1/MUe0WlmRgWG+imZsLnSSXIhOF12oNjVEl6flt68MOy1afi4CJTLjYwlo3/+t6xakRCaYIv/1XvAZbu/s3byDZ2A8kd4Bsctq0mqbfFP/awm+o6fSi6aQq7XSCqFDRwX4PP9EMLzwhRGJoNkOcAROZS3z+LycRSziJjpbi5F179Hh49EWCoF0AJcZuOfJ95RxH68E3c='


## Verification Phase

##### decryption

In [18]:
user_provided_password = "pavaniMandadi8"
BLOCK_SIZE = 16
unpad = lambda s: s[:-ord(s[len(s) - 1:])]
symbols = {"00": "0", "01": "1", "11": "*", "10": "*"}
symbols_reverse = {value:key for key, value in symbols.items()}

def decrypt(enc, password):
    private_key = hashlib.sha256(password.encode("utf-8")).digest()
    enc = base64.b64decode(enc)
    iv = enc[:16]
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(enc[16:]))

decrypted = decrypt(encoded_string, user_provided_password)
decrypted_string = bytes.decode(decrypted)
print("AES decrypted String :", decrypted_string)

AES decrypted String : **0001*1*01*0001****001*1*1***1**1**001*****1*01**00000001*01*01*0000000001**0001**1**1***01*01*1**1**1*01*1*1*001**1*1*1**001*001*1***000001*01***0001*001*1**01*1*01*00001****01**1*1*000001*0000001*001***1*1*001*1*1*********1*1*01*1***01***001**1**1*01**


In [37]:
sample = "**0001*1*01*0001****001*1*1***1**1**001*****1*01**00000001*01*01*0000000001**0001**1**1***01*01*1**1**1*01*1*1*001**1*1*1**001*001*1***000001*01***0001*001*1**01*1*01*00001****01**1*1*000001*0000001*001***1*1*001*1*1*********1*1*01*1***01***001**1**1*01**"
symbols_reverse

{'0': '00', '1': '01', '*': '10'}

In [38]:
sample_list = "".join([symbols_reverse[each] if each in ["1", "0"] else each for each in sample])
sample_list

'**00000001*01*0001*00000001****000001*01*01***01**01**000001*****01*0001**0000000000000001*0001*0001*00000000000000000001**00000001**01**01***0001*0001*01**01**01*0001*01*01*000001**01*01*01**000001*000001*01***000000000001*0001***00000001*000001*01**0001*01*0001*0000000001****0001**01*01*000000000001*00000000000001*000001***01*01*000001*01*01*********01*01*0001*01***0001***000001**01**01*0001**'

In [41]:
"1100001010010000111100010101110110110001111101001100000000100100100000000001100001101101110010010110110100101010001101010110001000101110000001001110000100010110010100100000111100110101000000100000001000111010100010101111111110101001011100111000110110100111"

'1100001010010000111100010101110110110001111101001100000000100100100000000001100001101101110010010110110100101010001101010110001000101110000001001110000100010110010100100000111100110101000000100000001000111010100010101111111110101001011100111000110110100111'